In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.types import StructType,StructField,IntegerType,FloatType,LongType,StringType
from reco_utils.common.spark_utils import start_or_get_spark
from reco_utils.dataset.spark_splitters import spark_random_split
from pyspark.sql.functions import regexp_replace, col


In [2]:
sc = pyspark.SparkContext()
sql = SQLContext(sc)

In [3]:
spark = SparkSession \
    .builder \
    .appName("ALS") \
    .getOrCreate()

In [4]:
spark.version

'3.1.1'

In [9]:
df = spark.read.csv('rating.csv').toDF('userId', 'productId','Rating','timestamp')


In [10]:
df.show(10)

+--------------+----------+------+----------+
|        userId| productId|Rating| timestamp|
+--------------+----------+------+----------+
| AKM1MP6P0OYPR|0132793040|   5.0|1365811200|
|A2CX7LUOHB2NDG|0321732944|   5.0|1341100800|
|A2NWSAGRHCP8N5|0439886341|   1.0|1367193600|
|A2WNBOD3WNDNKT|0439886341|   3.0|1374451200|
|A1GI0U4ZRJA8WN|0439886341|   1.0|1334707200|
|A1QGNMC6O1VW39|0511189877|   5.0|1397433600|
|A3J3BRHTDRFJ2G|0511189877|   2.0|1397433600|
|A2TY0BTJOTENPG|0511189877|   5.0|1395878400|
|A34ATBPOK6HCHY|0511189877|   5.0|1395532800|
| A89DO69P0XZ27|0511189877|   5.0|1395446400|
+--------------+----------+------+----------+
only showing top 10 rows



### Remove Alphabet from aphanumeric ISBN Columns using 'withColumn' function

In [11]:
df=df.withColumn("userId",regexp_replace(col("userId"),"[a-zA-Z]",""))


In [12]:
COL_USER = "userId"
COL_ITEM = "productId"
COL_RATING = "Rating"
COL_PREDICTION = "prediction"
COL_TIMESTAMP = "timestamp"
schema=StructType(
    (
    StructField(COL_USER,LongType()),
    StructField(COL_ITEM,LongType()),
    StructField(COL_RATING,FloatType()),
    StructField(COL_TIMESTAMP, LongType()),
    
    ))





In [13]:
df.first()

Row(userId='160', productId='0132793040', Rating='5.0', timestamp='1365811200')

<h2>Changing the schema of the Dataframe using 'withColumn' method</h2>

In [14]:
df2 = df.withColumn("userId",col("userId").cast(IntegerType())) \
    .withColumn("productId",col("productId").cast(IntegerType())) \
    .withColumn("Rating",col("Rating").cast(FloatType())) \
.withColumn("timestamp",col("timestamp").cast(LongType())) 

In [15]:
df.printSchema()

root
 |-- userId: string (nullable = true)
 |-- productId: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [16]:
df2.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- productId: integer (nullable = true)
 |-- Rating: float (nullable = true)
 |-- timestamp: long (nullable = true)



In [18]:
df2.select('userId').show(5)

+------+
|userId|
+------+
|   160|
|   272|
|   285|
|    23|
|  1048|
+------+
only showing top 5 rows



In [19]:
df2.show(4)

+------+---------+------+----------+
|userId|productId|Rating| timestamp|
+------+---------+------+----------+
|   160|132793040|   5.0|1365811200|
|   272|321732944|   5.0|1341100800|
|   285|439886341|   1.0|1367193600|
|    23|439886341|   3.0|1374451200|
+------+---------+------+----------+
only showing top 4 rows



In [15]:
df2.describe().show()

+-------+------------------+--------------------+------------------+
|summary|           User-ID|                ISBN|       Book-Rating|
+-------+------------------+--------------------+------------------+
|  count|           1149780|             1062289|           1149780|
|   mean|140386.39512602412|5.3162145050241506E8|2.8669501991685364|
| stddev| 80562.27771851176| 3.929768082443173E8| 3.854183859201656|
|    min|                 2|                   0|               0.0|
|    max|            278854|          2130530508|              10.0|
+-------+------------------+--------------------+------------------+



In [20]:
from pyspark.sql.functions import isnan, when, count, col

df2.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+------+---------+------+---------+
|userId|productId|Rating|timestamp|
+------+---------+------+---------+
|     0|        0|     0|        0|
+------+---------+------+---------+



<h1 style="text-align:center">Runnning SQL queries Programmatically</h1>

In [23]:
df.createOrReplaceTempView("rating")

In [24]:
sqlDF=spark.sql("select `userId`,count(*) as Frequency from rating group by `userId`")

In [25]:
sqlDF.show()

+-------+---------+
| userId|Frequency|
+-------+---------+
| 330499|        7|
|   2904|      545|
| 121080|        4|
|  17506|       35|
| 184356|       17|
| 239830|        9|
|   1572|      595|
|   2294|      499|
|3658917|        1|
|  24269|       42|
|  18726|       61|
|   1090|      544|
|   2088|      628|
|     07|     3476|
|    296|     4478|
|  20512|       43|
|   2162|      624|
|1106193|        4|
|    467|      533|
|2898472|        1|
+-------+---------+
only showing top 20 rows



In [26]:
df2=df2.dropna()


<h1 style="color:red;text-align:center;">Simple Algorithm for Recommendation (SAR)</h1>

In [27]:
from pyspark.ml.recommendation import ALS

In [28]:
# dataset split into training and testing set
(training, test) = df2.randomSplit([0.8, 0.2])
# training the model
als = ALS(maxIter=5, implicitPrefs=True,userCol="userId", itemCol="productId", ratingCol="Rating",coldStartStrategy="drop")
model = als.fit(training)
# predict using the testing datatset
predictions = model.transform(test)
predictions.show()

+------+---------+------+----------+-----------+
|userId|productId|Rating| timestamp| prediction|
+------+---------+------+----------+-----------+
|  1088|972683275|   4.0|1377907200|   1.023531|
|   243|972683275|   4.0|1352764800|  0.9404261|
|   322|972683275|   5.0|1348444800|  0.9404261|
|    34|972683275|   4.0|1395705600| 0.96759874|
|    26|972683275|   5.0|1377388800| 0.96618086|
|   332|972683275|   4.0|1360108800|0.034492627|
|    22|972683275|   5.0|1324166400|  1.0609893|
|   372|972683275|   5.0|1372377600|0.021905364|
|   346|972683275|   5.0|1384646400|  0.8606039|
|   224|972683275|   1.0|1344124800|   1.001984|
|   224|972683275|   5.0|1390003200|   1.001984|
|   259|972683275|   4.0|1295654400|  0.9404261|
|   182|972683275|   5.0|1378857600|  0.8403057|
|     6|972683275|   4.0|1283126400| 0.80649614|
|    16|972683275|   4.0|1396742400|  1.0118737|
|   142|972683275|   5.0|1366156800| 0.86027867|
|    20|972683275|   5.0|1355529600|  0.3661771|
|   295|972683275|  

In [29]:

test.first()

Row(User-ID=2, ISBN=195153448, Book-Rating=0.0)

In [118]:
df2.count()

1062289

In [119]:
training.show()


+-------+----------+-----------+
|User-ID|      ISBN|Book-Rating|
+-------+----------+-----------+
|      2| 195153448|          0|
|      7|  34542252|          0|
|      8|   2005018|          5|
|      8|  74322678|          5|
|      8|  80652121|          0|
|      8| 374157065|          0|
|      8| 393045218|          0|
|      8| 425176428|          0|
|      8| 679425608|          0|
|      8| 887841740|          5|
|      8|1552041778|          5|
|      8|1558746218|          0|
|      8|1567407781|          6|
|      8|1575663937|          6|
|      8|1881320189|          7|
|      9| 440234743|          0|
|     10|1841721522|          0|
|     12|1879384493|         10|
|     14|  61076031|          5|
|     14| 439095026|          5|
+-------+----------+-----------+
only showing top 20 rows



In [120]:
sc.setCheckpointDir('checkpoint/')
ALS.checkpointInterval = 2

In [ ]:
df_2=sc.textFile("rating.csv")

In [5]:
spark = SQLContext(sc)                                                                             
df = spark.createDataFrame([(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],["id", "category"])

In [8]:
stringIndexer = StringIndexer(inputCol="category", outputCol="categoryIndex")

In [9]:
model = stringIndexer.fit(df)

In [10]:
indexed = model.transform(df)

In [11]:
indexed.show()

+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+

